In [28]:
%matplotlib inline

In [29]:
import numpy as np
import keras

from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.models import Model

from keras.constraints import maxnorm

from keras.layers import Input
from keras.layers.core import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import BatchNormalization
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Activation

from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adam

from keras.preprocessing import image

from keras.utils.np_utils import to_categorical

from keras.datasets import cifar10

from utils import *

In [30]:
batch_size = 64 # 4096

In [31]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test )

In [56]:
mean_pixel_value = X_train.mean().astype(np.float32)
std_pixel_value  = X_train.std( ).astype(np.float32)

def normalize_input(X):
    return (X - mean_pixel_value) / std_pixel_value

In [57]:
augmentation_parameters = {
    'width_shift_range'  : 0.1,
    'height_shift_range' : 0.1,
    'horizontal_flip'    : True
}

augmented_generator = image.ImageDataGenerator(**augmentation_parameters)
batches_generator   = image.ImageDataGenerator()
batches             = batches_generator.flow(X_train, y_train, batch_size = batch_size)
augmented_batches   = augmented_generator.flow(X_train, y_train, batch_size = batch_size)
test_generator      = image.ImageDataGenerator()
test_batches        = test_generator.flow(X_test , y_test , batch_size = batch_size)

In [37]:
def get_deep_cnn(lr = 0.01, nb_epoch = 25):
    model = Sequential([
            Lambda(normalize_input, input_shape = (3, 32, 32)),
            
            Convolution2D(32, 3, 3, activation = 'relu', border_mode = 'same'),
            Dropout(0.2),
            Convolution2D(32, 3, 3, activation = 'relu', border_mode = 'same'),
            MaxPooling2D(pool_size = (2, 2)),
            
            Convolution2D(64, 3, 3, activation = 'relu', border_mode = 'same'),
            Dropout(0.2),
            Convolution2D(64, 3, 3, activation = 'relu', border_mode = 'same'),
            MaxPooling2D(pool_size = (2, 2)),
            
            Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
            Dropout(0.2),
            Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
            MaxPooling2D(pool_size = (2, 2)),
            
            
            Flatten(),
            
            Dropout(0.2),
            Dense(1024, activation = 'relu', W_constraint = maxnorm(3)),
            Dropout(0.2),
            Dense(512, activation = 'relu', W_constraint = maxnorm(3)),
            Dropout(0.2),
            
            Dense(10, activation = 'softmax')
        ])
    decay = lr / nb_epoch
    sgd = SGD(lr = lr, momentum = 0.9, decay = decay, nesterov = False)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    return model

In [39]:
cnn = get_deep_cnn()

In [41]:
fg_parameters = {
    'generator'         : batches,
    'samples_per_epoch' : batches.N,
    'nb_epoch'          : 25,
    'validation_data'   : test_batches,
    'nb_val_samples'    : test_batches.N
}

In [42]:
cnn.fit_generator(**fg_parameters)

Epoch 1/25
50000/50000 [==============================] - 30s - loss: 1.7146 - acc: 0.3683 - val_loss: 1.3817 - val_acc: 0.4950
Epoch 2/25
50000/50000 [==============================] - 30s - loss: 1.2709 - acc: 0.5359 - val_loss: 1.1030 - val_acc: 0.6096
Epoch 3/25
50000/50000 [==============================] - 30s - loss: 1.0506 - acc: 0.6227 - val_loss: 0.9454 - val_acc: 0.6650
Epoch 4/25
50000/50000 [==============================] - 30s - loss: 0.9195 - acc: 0.6741 - val_loss: 0.9046 - val_acc: 0.6864
Epoch 5/25
50000/50000 [==============================] - 30s - loss: 0.8191 - acc: 0.7100 - val_loss: 0.7459 - val_acc: 0.7415
Epoch 6/25
50000/50000 [==============================] - 29s - loss: 0.7418 - acc: 0.7395 - val_loss: 0.7067 - val_acc: 0.7539
Epoch 7/25
50000/50000 [==============================] - 29s - loss: 0.6942 - acc: 0.7554 - val_loss: 0.6661 - val_acc: 0.7698
Epoch 8/25
50000/50000 [==============================] - 29s - loss: 0.6486 - acc: 0.7719 - val_loss: 0

In [48]:
def get_deep_cnn_batchnorm(lr = 0.01, nb_epoch = 25):
    model = Sequential([
            Lambda(normalize_input, input_shape = (3, 32, 32)),
            
            Convolution2D(32, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            Dropout(0.2),
            Convolution2D(32, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            MaxPooling2D(pool_size = (2, 2)),
            
            Convolution2D(64, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            Dropout(0.2),
            Convolution2D(64, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            MaxPooling2D(pool_size = (2, 2)),
            
            Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            Dropout(0.2),
            Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
            BatchNormalization(axis = 1),
            MaxPooling2D(pool_size = (2, 2)),
            
            
            Flatten(),
            
            Dropout(0.2),
            Dense(1024, activation = 'relu', W_constraint = maxnorm(3)),
            BatchNormalization(),
            Dropout(0.2),
            Dense(512, activation = 'relu', W_constraint = maxnorm(3)),
            BatchNormalization(),
            Dropout(0.2),
            
            Dense(10, activation = 'softmax')
        ])
    decay = lr / nb_epoch
    sgd = SGD(lr = lr, momentum = 0.9, decay = decay, nesterov = False)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    return model

In [49]:
cnn_batchnorm = get_deep_cnn_batchnorm()

In [50]:
fg_parameters = {
    'generator'         : batches,
    'samples_per_epoch' : batches.N,
    'nb_epoch'          : 25,
    'validation_data'   : test_batches,
    'nb_val_samples'    : test_batches.N
}

In [52]:
cnn_batchnorm.fit_generator(**fg_parameters)

Epoch 1/25
50000/50000 [==============================] - 74s - loss: 1.6694 - acc: 0.4206 - val_loss: 1.2820 - val_acc: 0.5397
Epoch 2/25
50000/50000 [==============================] - 72s - loss: 1.1958 - acc: 0.5744 - val_loss: 0.9900 - val_acc: 0.6510
Epoch 3/25
50000/50000 [==============================] - 72s - loss: 0.9637 - acc: 0.6584 - val_loss: 0.8294 - val_acc: 0.7120
Epoch 4/25
50000/50000 [==============================] - 72s - loss: 0.8218 - acc: 0.7098 - val_loss: 0.7600 - val_acc: 0.7365
Epoch 5/25
50000/50000 [==============================] - 72s - loss: 0.7401 - acc: 0.7402 - val_loss: 0.7079 - val_acc: 0.7578
Epoch 6/25
50000/50000 [==============================] - 71s - loss: 0.6690 - acc: 0.7612 - val_loss: 0.6655 - val_acc: 0.7672
Epoch 7/25
50000/50000 [==============================] - 71s - loss: 0.6140 - acc: 0.7821 - val_loss: 0.6180 - val_acc: 0.7897
Epoch 8/25
50000/50000 [==============================] - 71s - loss: 0.5818 - acc: 0.7945 - val_loss: 0

In [53]:
cnn_batchnorm_augmentation = get_deep_cnn_batchnorm()

In [59]:
fg_parameters = {
    'generator'         : augmented_batches,
    'samples_per_epoch' : augmented_batches.N,
    'nb_epoch'          : 25,
    'validation_data'   : test_batches,
    'nb_val_samples'    : test_batches.N
}

In [60]:
cnn_batchnorm_augmentation.fit_generator(**fg_parameters)

Epoch 1/25
50000/50000 [==============================] - 74s - loss: 1.7279 - acc: 0.3985 - val_loss: 1.3020 - val_acc: 0.5398
Epoch 2/25
50000/50000 [==============================] - 72s - loss: 1.2696 - acc: 0.5494 - val_loss: 1.0116 - val_acc: 0.6379
Epoch 3/25
50000/50000 [==============================] - 72s - loss: 1.0679 - acc: 0.6233 - val_loss: 0.9033 - val_acc: 0.6807
Epoch 4/25
50000/50000 [==============================] - 72s - loss: 0.9471 - acc: 0.6662 - val_loss: 0.7717 - val_acc: 0.7316
Epoch 5/25
50000/50000 [==============================] - 72s - loss: 0.8562 - acc: 0.6985 - val_loss: 0.7094 - val_acc: 0.7510
Epoch 6/25
50000/50000 [==============================] - 72s - loss: 0.8012 - acc: 0.7163 - val_loss: 0.6809 - val_acc: 0.7614
Epoch 7/25
50000/50000 [==============================] - 72s - loss: 0.7590 - acc: 0.7322 - val_loss: 0.6248 - val_acc: 0.7836
Epoch 8/25
50000/50000 [==============================] - 72s - loss: 0.7236 - acc: 0.7452 - val_loss: 0

In [61]:
cnn_batchnorm_augmentation.fit_generator(**fg_parameters)

Epoch 1/25
50000/50000 [==============================] - 74s - loss: 0.4982 - acc: 0.8253 - val_loss: 0.4469 - val_acc: 0.8450
Epoch 2/25
50000/50000 [==============================] - 72s - loss: 0.4917 - acc: 0.8274 - val_loss: 0.4477 - val_acc: 0.8490
Epoch 3/25
50000/50000 [==============================] - 72s - loss: 0.4852 - acc: 0.8291 - val_loss: 0.4497 - val_acc: 0.8493
Epoch 4/25
50000/50000 [==============================] - 72s - loss: 0.4831 - acc: 0.8311 - val_loss: 0.4480 - val_acc: 0.8438
Epoch 5/25
50000/50000 [==============================] - 72s - loss: 0.4743 - acc: 0.8333 - val_loss: 0.4421 - val_acc: 0.8496
Epoch 6/25
50000/50000 [==============================] - 72s - loss: 0.4697 - acc: 0.8352 - val_loss: 0.4311 - val_acc: 0.8534
Epoch 7/25
50000/50000 [==============================] - 72s - loss: 0.4689 - acc: 0.8344 - val_loss: 0.4321 - val_acc: 0.8524
Epoch 8/25
50000/50000 [==============================] - 72s - loss: 0.4692 - acc: 0.8355 - val_loss: 0

In [63]:
cnn_batchnorm_augmentation.save_weights('models/cifar10_batchnorm_augmentation.h5')

In [65]:
cnn_batchnorm_augmentation.load_weights('models/cifar10_batchnorm_augmentation.h5')